In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("galaxy_users.csv")
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No


### Q1.

In [3]:
df_q1 = df.loc[:, "OnlineSecurity":"StreamingMovies"].copy()
df_q1.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,No,No,No
1,Yes,No,Yes,No,No,No


In [4]:
df_q1 = df_q1.replace({"Yes": 1, "No": 0})
df_q1.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0,1,0,0,0,0
1,1,0,1,0,0,0


In [5]:
df_q1["OnlineSecurity"].unique()

array([0, 1, 'No internet service'], dtype=object)

In [6]:
[df_q1.iloc[:, 0].unique(),
 df_q1.iloc[:, 1].unique(),
 df_q1.iloc[:, 2].unique(), 
 df_q1.iloc[:, 3].unique(),
 df_q1.iloc[:, 4].unique(),
 df_q1.iloc[:, 5].unique()]

[array([0, 1, 'No internet service'], dtype=object),
 array([1, 0, 'No internet service'], dtype=object),
 array([0, 1, 'No internet service'], dtype=object),
 array([0, 1, 'No internet service'], dtype=object),
 array([0, 1, 'No internet service'], dtype=object),
 array([0, 1, 'No internet service'], dtype=object)]

In [7]:
for n in range(df_q1.shape[1]):
    print(df_q1.columns[n], df_q1.iloc[:, n].unique())

OnlineSecurity [0 1 'No internet service']
OnlineBackup [1 0 'No internet service']
DeviceProtection [0 1 'No internet service']
TechSupport [0 1 'No internet service']
StreamingTV [0 1 'No internet service']
StreamingMovies [0 1 'No internet service']


In [8]:
# df_q1.unique()
# df_q1.drop_duplicates()
# df_q1.apply(lambda x: [x.unique()]) # 시험버전(리스트로 감싸주어야 한다.)
df_q1.apply(lambda x: x.unique())

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0,1,0,0,0,0
1,1,0,1,1,1,1
2,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service


In [9]:
df_q1_sub = df_q1.loc[df_q1["OnlineSecurity"] != "No internet service", ]
df_q1_sub.apply(lambda x: x.unique())

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0,1,0,0,0,0
1,1,0,1,1,1,1


In [10]:
len(df_q1_sub)

5512

In [11]:
df_q1.loc[df_q1["OnlineSecurity"] == "No internet service", ]

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
11,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
16,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
21,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
22,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
33,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
...,...,...,...,...,...,...
7006,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
7008,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
7009,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
7019,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service


In [12]:
df_q1_sub = df_q1.replace({"No internet service": np.nan}).dropna()
df_q1_sub.apply(lambda x: x.unique())

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,1.0,1.0,1.0


In [13]:
len(df_q1_sub)

5512

In [14]:
serv_cnt = df_q1_sub.sum(axis = 1).value_counts()
serv_cnt

3.0    1117
2.0    1033
1.0     966
4.0     850
0.0     693
5.0     569
6.0     284
Name: count, dtype: int64

In [15]:
round(serv_cnt[1] / serv_cnt[6], 2)

3.4

In [16]:
serv_cnt.iloc[1]

1033

In [17]:
df_1_sub = df_q1_sub.copy()
df_1_sub.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0


In [18]:
df_1_sub["total"] = df_1_sub.sum(axis = 1)
df_1_sub.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,total
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0,0.0,2.0


### Q2.

In [19]:
df_q2 = df[["tenure", "MonthlyCharges", "TotalCharges"]].copy()
df_q2.head(2)

,tenure,MonthlyCharges,TotalCharges
0,1,29.85,29.85
1,34,56.95,1889.50


In [21]:
7 // 2, 14 // 5

(3, 2)

In [22]:
df_q2["month"] = df_q2["TotalCharges"] // df_q2["MonthlyCharges"]
df_q2.head(2)

,tenure,MonthlyCharges,TotalCharges,month
0,1,29.85,29.85,1.0
1,34,56.95,1889.50,33.0


In [23]:
df_q2.iloc[:, [0, 1, 3]].corr().round(3) # 0.999

,tenure,MonthlyCharges,month
tenure,1.000,0.247,0.999
MonthlyCharges,0.247,1.000,0.246
month,0.999,0.246,1.000


### Q3.

In [27]:
col1 = ["SeniorCitizen", "Partner", "Dependents", "tenure", "MonthlyCharges", "TotalCharges"]
col2 = ["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingMovies", "PaperlessBilling"]
df_q3 = df[["Churn"] + col1 + col2].copy()

In [28]:
df_q3 = df_q3.replace({"Yes": 1, "No": 0})

In [30]:
df_q3.dtypes

Churn                 int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
MonthlyCharges      float64
TotalCharges        float64
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingMovies      object
PaperlessBilling      int64
dtype: object

In [ ]:
df_q3_cat = df_q3.loc[:, df_q3.dtypes == "object"]
df_q3_cat.head(2)

In [38]:
df_q3.loc[:2, df_q3.columns.str.contains("^Online")]

,OnlineSecurity,OnlineBackup
0,0,1
1,1,0
2,1,1


In [34]:
df_q3_cat = df_q3.select_dtypes(exclude = "number") # 시험버전에서 버그로 동작을 하지 않음.
df_q3_cat.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies
0,0,1,0,0,0
1,1,0,1,0,0


In [35]:
df_q3_cat.apply(lambda x: x.unique())

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies
0,0,1,0,0,0
1,1,0,1,1,1
2,No internet service,No internet service,No internet service,No internet service,No internet service


In [36]:
df_q3 = df_q3.replace("No internet service", -1)

In [37]:
df_q3.head(2)

,Churn,SeniorCitizen,Partner,Dependents,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,PaperlessBilling
0,0,0,1,0,1,29.85,29.85,0,1,0,0,0,1
1,0,0,0,0,34,56.95,1889.50,1,0,1,0,0,0


In [39]:
df_train, df_test = train_test_split(df_q3, train_size = 0.7, random_state = 123)
len(df_train), len(df_test)

(4922, 2110)

In [40]:
model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor  = model_nor.transform(df_test)

In [ ]:
arr_train_nor[:2, ]

In [42]:
df_train_nor = pd.DataFrame(arr_train_nor, columns = df_train.columns)
df_train_nor.head(2)

,Churn,SeniorCitizen,Partner,Dependents,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,PaperlessBilling
0,1.0,0.0,0.0,0.0,0.084507,0.811161,0.075519,0.5,1.0,0.5,0.5,1.0,1.0
1,1.0,0.0,1.0,0.0,0.000000,0.607374,0.006987,0.5,0.5,0.5,0.5,0.5,1.0


In [45]:
model_lr = LogisticRegression(random_state = 123)
model_lr.fit(X = arr_train_nor[:, 1:],
             y = arr_train_nor[:, 0 ])
pred = model_lr.predict(arr_test_nor[:, 1:])
pred[:4]

array([0., 0., 0., 0.])

In [46]:
round(f1_score(y_true = arr_test_nor[:, 0], 
               y_pred = pred), 2)

0.55

#### Q. 범주형 변수의 특정 범주를 제외한 나머지 범주를 원하는 (단일)값으로 치환하고자 하는 경우.
※ H와 G를 제외한 나머지 범주는 모두 -1로 치환

In [47]:
df_dia = pd.read_csv("../diamonds.csv")
df_dia.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [49]:
# df_dia.iloc[:, 1:4].apply(lambda x: [x.unique()]) # 시험버전
df_dia.iloc[:, 1:4].apply(lambda x: x.unique())

cut         [Ideal, Premium, Good, Very Good, Fair]
color                         [E, I, J, H, F, G, D]
clarity    [SI2, SI1, VS1, VS2, VVS2, VVS1, I1, IF]
dtype: object

In [52]:
ser_u = df_dia.iloc[:, 1:4].apply(lambda x: x.unique()).explode()
ser_repl = pd.Series(np.where(ser_u.isin(["H", "G"]), ser_u, -1),
                     index = ser_u)
ser_repl.to_dict()

{'Ideal': -1,
 'Premium': -1,
 'Good': -1,
 'Very Good': -1,
 'Fair': -1,
 'E': -1,
 'I': -1,
 'J': -1,
 'H': 'H',
 'F': -1,
 'G': 'G',
 'D': -1,
 'SI2': -1,
 'SI1': -1,
 'VS1': -1,
 'VS2': -1,
 'VVS2': -1,
 'VVS1': -1,
 'I1': -1,
 'IF': -1}

In [53]:
df_dia = df_dia.replace(ser_repl)
df_dia.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,-1,-1,-1,61.5,55.0,326,3.95,3.98,2.43
1,0.21,-1,-1,-1,59.8,61.0,326,3.89,3.84,2.31


In [54]:
df_dia.iloc[:, 1:4].apply(lambda x: x.unique())

cut              [-1]
color      [-1, H, G]
clarity          [-1]
dtype: object